In [27]:
from datetime import datetime
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from tqdm import tqdm

from OddsAPI import API
api = API()
api.setAPI("6rySBr2yntSdYTsLeYvthIEi554gzp1R97TUiOMjn0")

In [28]:
pd.set_option('display.max_rows', 100)

In [62]:
## pivot: converts a dataframe that contains two seperate rows for each player (one for over, one for under) to have one row for each player
def pivot(df):
    df = df.copy()
    pivot_df = pd.pivot_table(df, values='odds', index=['handicap', 'participant_name'], columns='name').reset_index()
    try:
        pivot_df.columns = ['line', 'participant_name', 'over_odds', 'under_odds']
    except Exception as e:

        print("\n\n PIVOT OUTPUT")
        print(e)
        print(df)
        print(pivot_df)
        print(pivot_df.columns)
    return pivot_df

## calculate odds
## adapted from Ammar Sulmanjee
def calculate_odds(x, y=None):
    if not y:
        if x>= 0: return 1 + x/100
        return 1 + 100/abs(x)

    imp_prob1 = (1 / calculate_odds(x)) * 100
    imp_prob2 = (1 / calculate_odds(y)) * 100

    total_implied_prob = round(imp_prob1 + imp_prob2, 4)
    fair_prob1 = round(imp_prob1 / total_implied_prob * 100, 2)
    fair_prob2 = round(imp_prob2 / total_implied_prob * 100, 2)

    return [fair_prob1, fair_prob2]

## saves best bets
def download_bets(best_bets):
    best_bets['market'] = best_bets['market'].str.replace(r'_|player|over_under', '', regex=True).str.strip()
    best_bets = best_bets.drop(columns=['game'])
    best_bets.to_csv(f"Bets_{datetime.now().strftime('%Y-%m-%d')}.csv", index=None)

## check palyer's raw data (debugging function)
def check_player(raw_bet_data, name, market, line=None):
    pd.set_option('display.max_rows', None)
    if line:
        display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market) & (raw_bet_data['handicap'] == line)])
        pd.reset_option('display.max_rows')
        return
    display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market)])
    pd.reset_option('display.max_rows')

def check_participants(df):
    for participant in df['participant_name'].unique():
        occurrences = df[df['participant_name'] == participant].shape[0]

        if occurrences == 2:
            continue
        return False
    return True


In [30]:
### Pull Underdog Lines
ud = api.get_fantasy_lines()

markets = set(ud['market'])

games = api.get_nba_games()
games = [i['game_id'] for i in games['games']]

In [65]:
bets = pd.DataFrame(columns = ['participant_name', 'market', 'line', 'over_odds', 'under_odds'])
raw_bet_data = pd.DataFrame()

for game in tqdm(games, desc="Processing games"):
    for market in markets:
            
        # get sportsbook odds
        try:
            bookies_data = api.get_most_recent_odds(game, market)
        except Exception as e:
            if str(e) == "ERROR 422": continue
            else: 
                print(e)
                continue
             
        books = []
        
        for i in range(len(bookies_data['sportsbooks'])):
            df = pd.DataFrame.from_dict(bookies_data['sportsbooks'][i]['market']['outcomes'])
            df = df[df['timestamp'].str.contains(datetime.now().strftime('%Y-%m-%d'))]

            df['book'] = bookies_data['sportsbooks'][i]['bookie_key']
            df['market'] = market
            df['name'] = df['name'].apply(lambda x: "over" if "over" in x.lower() else "under")


            books.append(df)
        try:
            raw_data = pd.concat(books)
            raw_bet_data = pd.concat([raw_bet_data, raw_data])
        except Exception as e: pass

        # final = pd.concat(books, axis=0)
        

        # concat the books into one
        new_datasets = []
        for dataset in books:
            dataset = pd.merge(dataset[['handicap', 'odds', 'participant_name', 'name', 'market']], ud, how='inner', on = ['participant_name', 'handicap' , 'market'])
            dataset = dataset[['handicap', 'odds', 'participant_name', 'name']]
            
            if dataset.shape[0] > 0 and dataset.shape[1] > 0 and check_participants(dataset):
                dataset = pivot(dataset)
                
                if dataset.shape[0] > 0 and dataset.shape[1] > 0:
                    new_datasets.append(dataset)
        if new_datasets == []: continue

        
        final = pd.concat(new_datasets, axis=0)
        final = final.dropna()

    
        final[['over_odds', 'under_odds']] = final.apply(lambda row: pd.Series(calculate_odds(row['over_odds'], row['under_odds'])), axis=1)
        final = final.groupby(['participant_name', 'line']).agg({'over_odds':'median', 'under_odds':'median'}).reset_index()


        final['ev'] = abs(final['over_odds'] - 50)
        final['market'] = market
        final['game'] = game


        if final.shape[0] > 0: 
            bets = pd.concat([bets, final[['participant_name', 'ev', 'market', 'line', 'over_odds', 'under_odds', 'game']]])

Processing games: 100%|██████████| 10/10 [00:51<00:00,  5.14s/it]


In [66]:
bets = bets.sort_values(by=['ev'], ascending=False)
bets = bets.dropna()
bets = bets.drop_duplicates(keep='first', ignore_index=True)

best_bets = bets.copy()
best_bets = best_bets[best_bets['ev'] > 4.5]

print(f"Total Bets found!: {bets.shape[0]}")
print(f"Total Positive EV Bets found!: {best_bets.shape[0]}")

Total Bets found!: 427
Total Positive EV Bets found!: 1


In [67]:
best_bets

,participant_name,market,line,over_odds,under_odds,ev,game
0,Nic Claxton,player_points_over_under,14.5,44.875,55.125,5.125,e9f33a7f036c43a32d2db5426f9bcc48


In [68]:
bets

,participant_name,market,line,over_odds,under_odds,ev,game
0,Nic Claxton,player_points_over_under,14.5,44.875,55.125,5.125,e9f33a7f036c43a32d2db5426f9bcc48
1,Kevin Durant,player_assists_rebounds_over_under,11.5,54.46,45.54,4.460,e9f33a7f036c43a32d2db5426f9bcc48
2,Tyler Herro,player_assists_points_over_under,22.5,54.34,45.66,4.340,c96521dde3008893e0f55e16327596c1
3,Brandon Ingram,player_threes_over_under,1.5,45.675,54.325,4.325,02e0bf8d3b5c84685db2a220fc0c97b5
4,Kevin Huerter,player_blocks_steals_over_under,1.5,45.73,54.27,4.270,c96521dde3008893e0f55e16327596c1
...,...,...,...,...,...,...,...
422,Jalen Duren,player_turnovers_over_under,1.5,50.0,50.0,0.000,35ad645e74f40025f5e9f33669f4f476
423,Damian Lillard,player_points_over_under,26.5,50.0,50.0,0.000,753b65609d24e43a89ef67b661c38b51
424,Deandre Ayton,player_points_over_under,14.5,50.0,50.0,0.000,753b65609d24e43a89ef67b661c38b51
425,Bojan Bogdanovic,player_assists_rebounds_over_under,5.5,50.0,50.0,0.000,35ad645e74f40025f5e9f33669f4f476


In [70]:
check_player(raw_bet_data, 'Nic Claxton', 'player_points_over_under', 14.5)

,timestamp,handicap,odds,participant,participant_name,name,description,book,market
7,2024-01-31T15:00:15,14.5,102,16552,Nic Claxton,over,Over Nic Claxton (Points),pinnacle,player_points_over_under
15,2024-01-31T15:00:15,14.5,-135,16552,Nic Claxton,under,Under Nic Claxton (Points),pinnacle,player_points_over_under
59,2024-01-31T14:44:12,14.5,114,16552,Nic Claxton,over,Over - Nic Claxton Points scored by the player...,betrivers,player_points_over_under
121,2024-01-31T14:44:12,14.5,-155,16552,Nic Claxton,under,Under - Nic Claxton Points scored by the playe...,betrivers,player_points_over_under
